# 🏠 Analyse Descriptive des Données Immobilières DVF (2020-2025)
## Projet Dashboard Immobilier - Anaïs

---

## 📚 Importation des bibliothèques

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10
%matplotlib inline

print("✅ Bibliothèques importées avec succès")

## 📥 1. Chargement et Nettoyage des Données

In [ ]:
# Chargement des données par année
foncieres_20 = pd.read_csv("ValeursFoncieres-2020.txt", sep="|", low_memory=False)
foncieres_21 = pd.read_csv("ValeursFoncieres-2021.txt", sep="|", low_memory=False)
foncieres_22 = pd.read_csv("ValeursFoncieres-2022.txt", sep="|", low_memory=False)
foncieres_23 = pd.read_csv("ValeursFoncieres-2023.txt", sep="|", low_memory=False)
foncieres_24 = pd.read_csv("ValeursFoncieres-2024.txt", sep="|", low_memory=False)
foncieres_25 = pd.read_csv("ValeursFoncieres-2025.txt", sep="|", low_memory=False)

print(f"📊 Données chargées:")
print(f"   2020: {len(foncieres_20):,} lignes")
print(f"   2021: {len(foncieres_21):,} lignes")
print(f"   2022: {len(foncieres_22):,} lignes")
print(f"   2023: {len(foncieres_23):,} lignes")
print(f"   2024: {len(foncieres_24):,} lignes")
print(f"   2025: {len(foncieres_25):,} lignes")

In [ ]:
# Concaténation de toutes les années
foncieres_all = pd.concat(
    [foncieres_20, foncieres_21, foncieres_22, foncieres_23, foncieres_24, foncieres_25], 
    axis=0, 
    ignore_index=True
)

print(f"📊 Dataset complet: {len(foncieres_all):,} lignes × {len(foncieres_all.columns)} colonnes")
print(f"💾 Taille en mémoire: {foncieres_all.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

In [ ]:
# Analyse des valeurs manquantes
na_count = foncieres_all.isna().sum()
na_percent = (na_count / len(foncieres_all)) * 100

na_summary = pd.DataFrame({
    'Colonnes': na_count.index,
    'Valeurs manquantes': na_count.values,
    'Pourcentage': na_percent.values
}).sort_values('Valeurs manquantes', ascending=False)

print("📊 Top 15 colonnes avec le plus de valeurs manquantes:\n")
print(na_summary.head(15).to_string(index=False))

In [ ]:
# Suppression des colonnes avec plus de 1 million de NA
foncieres_all = foncieres_all.loc[:, na_count < 1_000_000]

print(f"✅ Après nettoyage: {len(foncieres_all.columns)} colonnes conservées")

## 🔧 2. Prétraitement et Feature Engineering

In [ ]:
df = foncieres_all.copy()

# 1) Date mutation -> datetime + année/mois
df["Date mutation"] = pd.to_datetime(df["Date mutation"], format="%d/%m/%Y", errors="coerce")
df["year"] = df["Date mutation"].dt.year
df["month"] = df["Date mutation"].dt.month
df["quarter"] = df["Date mutation"].dt.quarter

# 2) Valeur foncière -> numérique (virgule -> point)
df["Valeur fonciere"] = (df["Valeur fonciere"].astype(str)
    .str.replace(",", ".", regex=False)
    .str.replace(" ", "", regex=False))
df["Valeur fonciere"] = pd.to_numeric(df["Valeur fonciere"], errors="coerce")

# 3) Code département en string
df["Code departement"] = df["Code departement"].astype(str).str.strip()

# 4) Filtrage basique
df = df[df["year"].isin([2020, 2021, 2022, 2023, 2024, 2025])]
df = df[df["Valeur fonciere"].notna() & (df["Valeur fonciere"] > 0)]

print(f"✅ Données prétraitées: {len(df):,} transactions valides")

In [ ]:
# Suppression des outliers (1%-99%)
q_low, q_high = df["Valeur fonciere"].quantile([0.01, 0.99])
df_clean = df[(df["Valeur fonciere"] >= q_low) & (df["Valeur fonciere"] <= q_high)].copy()

print(f"📊 Distribution après nettoyage:")
print(f"   Transactions conservées: {len(df_clean):,} ({len(df_clean)/len(df)*100:.1f}%)")
print(f"   Min: {df_clean['Valeur fonciere'].min():,.0f}€")
print(f"   Max: {df_clean['Valeur fonciere'].max():,.0f}€")
print(f"   Médiane: {df_clean['Valeur fonciere'].median():,.0f}€")
print(f"   Moyenne: {df_clean['Valeur fonciere'].mean():,.0f}€")

## 📊 3. Agrégation des Données

In [ ]:
# Agrégation par département et année
agg_dvf = (df_clean
    .groupby(["Code departement", "year"], as_index=False)
    .agg(
        prix_median=("Valeur fonciere", "median"),
        prix_moyen=("Valeur fonciere", "mean"),
        prix_min=("Valeur fonciere", "min"),
        prix_max=("Valeur fonciere", "max"),
        prix_std=("Valeur fonciere", "std"),
        nb_transactions=("Valeur fonciere", "size")
    )
)

print("✅ Données agrégées par département et année")
print(f"   {len(agg_dvf)} lignes (département × année)")
agg_dvf.head(10)

In [ ]:
# Ajout du nom des départements
departements = {
    '01': 'Ain', '02': 'Aisne', '03': 'Allier', '04': 'Alpes-de-Haute-Provence',
    '05': 'Hautes-Alpes', '06': 'Alpes-Maritimes', '07': 'Ardèche', '08': 'Ardennes',
    '09': 'Ariège', '10': 'Aube', '11': 'Aude', '12': 'Aveyron',
    '13': 'Bouches-du-Rhône', '14': 'Calvados', '15': 'Cantal', '16': 'Charente',
    '17': 'Charente-Maritime', '18': 'Cher', '19': 'Corrèze', '21': 'Côte-d\'Or',
    '22': 'Côtes-d\'Armor', '23': 'Creuse', '24': 'Dordogne', '25': 'Doubs',
    '26': 'Drôme', '27': 'Eure', '28': 'Eure-et-Loir', '29': 'Finistère',
    '2A': 'Corse-du-Sud', '2B': 'Haute-Corse', '30': 'Gard', '31': 'Haute-Garonne',
    '32': 'Gers', '33': 'Gironde', '34': 'Hérault', '35': 'Ille-et-Vilaine',
    '36': 'Indre', '37': 'Indre-et-Loire', '38': 'Isère', '39': 'Jura',
    '40': 'Landes', '41': 'Loir-et-Cher', '42': 'Loire', '43': 'Haute-Loire',
    '44': 'Loire-Atlantique', '45': 'Loiret', '46': 'Lot', '47': 'Lot-et-Garonne',
    '48': 'Lozère', '49': 'Maine-et-Loire', '50': 'Manche', '51': 'Marne',
    '52': 'Haute-Marne', '53': 'Mayenne', '54': 'Meurthe-et-Moselle', '55': 'Meuse',
    '56': 'Morbihan', '57': 'Moselle', '58': 'Nièvre', '59': 'Nord',
    '60': 'Oise', '61': 'Orne', '62': 'Pas-de-Calais', '63': 'Puy-de-Dôme',
    '64': 'Pyrénées-Atlantiques', '65': 'Hautes-Pyrénées', '66': 'Pyrénées-Orientales', '67': 'Bas-Rhin',
    '68': 'Haut-Rhin', '69': 'Rhône', '70': 'Haute-Saône', '71': 'Saône-et-Loire',
    '72': 'Sarthe', '73': 'Savoie', '74': 'Haute-Savoie', '75': 'Paris',
    '76': 'Seine-Maritime', '77': 'Seine-et-Marne', '78': 'Yvelines', '79': 'Deux-Sèvres',
    '80': 'Somme', '81': 'Tarn', '82': 'Tarn-et-Garonne', '83': 'Var',
    '84': 'Vaucluse', '85': 'Vendée', '86': 'Vienne', '87': 'Haute-Vienne',
    '88': 'Vosges', '89': 'Yonne', '90': 'Territoire de Belfort', '91': 'Essonne',
    '92': 'Hauts-de-Seine', '93': 'Seine-Saint-Denis', '94': 'Val-de-Marne', '95': 'Val-d\'Oise'
}

agg_dvf['Departement'] = agg_dvf['Code departement'].map(departements)
print("✅ Noms des départements ajoutés")

## 📈 4. Statistiques Descriptives Globales

In [ ]:
# Statistiques par année
stats_annuelles = agg_dvf.groupby('year').agg({
    'prix_median': ['mean', 'median', 'std'],
    'prix_moyen': ['mean', 'median'],
    'nb_transactions': 'sum'
}).round(2)

print("📊 STATISTIQUES ANNUELLES")
print("=" * 80)
stats_annuelles

In [ ]:
# Top 15 départements par prix médian moyen
top_depts = agg_dvf.groupby('Departement').agg({
    'prix_median': 'mean',
    'prix_moyen': 'mean',
    'nb_transactions': 'sum'
}).sort_values('prix_median', ascending=False).head(15)

print("\n🏆 TOP 15 DÉPARTEMENTS PAR PRIX MÉDIAN")
print("=" * 80)
top_depts.style.format({
    'prix_median': '{:,.0f}€',
    'prix_moyen': '{:,.0f}€',
    'nb_transactions': '{:,}'
})

In [ ]:
# Calcul de la croissance totale
evolution_globale = agg_dvf.groupby('year')['prix_median'].mean()
croissance_totale = ((evolution_globale.iloc[-1] / evolution_globale.iloc[0]) - 1) * 100
croissance_annuelle = croissance_totale / 5

print(f"\n💹 ÉVOLUTION DES PRIX (2020-2025)")
print("=" * 80)
print(f"Prix médian 2020: {evolution_globale.iloc[0]:,.0f}€")
print(f"Prix médian 2025: {evolution_globale.iloc[-1]:,.0f}€")
print(f"Croissance totale: +{croissance_totale:.2f}%")
print(f"Croissance annuelle moyenne: +{croissance_annuelle:.2f}%")

## 📊 5. Visualisations - Évolution Temporelle

In [ ]:
# Figure 1: Évolution des prix médians et moyens
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Évolution globale
evolution_prix = agg_dvf.groupby('year').agg({
    'prix_median': 'mean',
    'prix_moyen': 'mean'
})

ax1.plot(evolution_prix.index, evolution_prix['prix_median'], 
         marker='o', linewidth=3, markersize=10, label='Prix Médian', color='steelblue')
ax1.plot(evolution_prix.index, evolution_prix['prix_moyen'], 
         marker='s', linewidth=3, markersize=10, label='Prix Moyen', color='coral')
ax1.set_title('Évolution des Prix Immobiliers en France (2020-2025)', 
              fontsize=14, fontweight='bold', pad=20)
ax1.set_xlabel('Année', fontsize=12)
ax1.set_ylabel('Prix (€)', fontsize=12)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1000:.0f}k€'))

# Volume de transactions
transactions = agg_dvf.groupby('year')['nb_transactions'].sum()
bars = ax2.bar(transactions.index, transactions.values, color='steelblue', alpha=0.7, edgecolor='navy')
ax2.set_title('Volume Total de Transactions par Année', 
              fontsize=14, fontweight='bold', pad=20)
ax2.set_xlabel('Année', fontsize=12)
ax2.set_ylabel('Nombre de transactions', fontsize=12)
ax2.grid(True, alpha=0.3, axis='y')

# Ajout des valeurs sur les barres
for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:,.0f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Figure 2: Distribution des prix par année (Box Plot)
fig, ax = plt.subplots(figsize=(14, 6))

box_data = [agg_dvf[agg_dvf['year'] == year]['prix_median'].values 
            for year in sorted(agg_dvf['year'].unique())]

bp = ax.boxplot(box_data, labels=sorted(agg_dvf['year'].unique()),
                patch_artist=True, notch=True, showmeans=True)

# Coloration des boîtes
colors = plt.cm.viridis(np.linspace(0, 1, len(box_data)))
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax.set_title('Distribution des Prix Médians par Année', 
             fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('Année', fontsize=12)
ax.set_ylabel('Prix Médian (€)', fontsize=12)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1000:.0f}k€'))
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 🗺️ 6. Visualisations - Comparaisons Géographiques

In [ ]:
# Top 15 départements les plus chers (2025)
top_dept_2025 = agg_dvf[agg_dvf['year'] == 2025].nlargest(15, 'prix_median')

fig, ax = plt.subplots(figsize=(12, 8))

bars = ax.barh(range(len(top_dept_2025)), top_dept_2025['prix_median'], 
               color=plt.cm.RdYlGn_r(np.linspace(0.3, 0.9, len(top_dept_2025))),
               edgecolor='black', linewidth=0.5)

ax.set_yticks(range(len(top_dept_2025)))
ax.set_yticklabels(top_dept_2025['Departement'].values)
ax.set_xlabel('Prix Médian (€)', fontsize=12)
ax.set_title('Top 15 Départements - Prix Médian 2025', 
             fontsize=14, fontweight='bold', pad=20)
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1000:.0f}k€'))
ax.invert_yaxis()
ax.grid(True, alpha=0.3, axis='x')

# Ajout des valeurs
for i, (idx, row) in enumerate(top_dept_2025.iterrows()):
    ax.text(row['prix_median'], i, f" {row['prix_median']:,.0f}€", 
            va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Scatter plot: Prix vs Transactions (2025)
data_2025 = agg_dvf[agg_dvf['year'] == 2025].copy()

fig, ax = plt.subplots(figsize=(14, 8))

scatter = ax.scatter(data_2025['nb_transactions'], 
                     data_2025['prix_median'],
                     s=data_2025['prix_median']/500,
                     c=data_2025['prix_median'],
                     cmap='viridis',
                     alpha=0.6,
                     edgecolors='black',
                     linewidth=0.5)

# Annotation des départements les plus remarquables
top_5_prix = data_2025.nlargest(5, 'prix_median')
for idx, row in top_5_prix.iterrows():
    ax.annotate(row['Departement'], 
                (row['nb_transactions'], row['prix_median']),
                fontsize=9, alpha=0.8,
                xytext=(5, 5), textcoords='offset points')

ax.set_title('Prix Médian vs Volume de Transactions (2025)', 
             fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('Nombre de Transactions', fontsize=12)
ax.set_ylabel('Prix Médian (€)', fontsize=12)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1000:.0f}k€'))
ax.grid(True, alpha=0.3)

cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Prix Médian (€)', fontsize=11)

plt.tight_layout()
plt.show()

## 🔥 7. Heatmap - Évolution des Prix par Département

In [ ]:
# Sélection des 20 départements les plus actifs
top_20_depts = (agg_dvf.groupby('Departement')['nb_transactions']
                .sum()
                .nlargest(20)
                .index)

# Création du pivot table
heatmap_data = agg_dvf[agg_dvf['Departement'].isin(top_20_depts)].pivot_table(
    values='prix_median',
    index='Departement',
    columns='year'
)

# Heatmap
fig, ax = plt.subplots(figsize=(12, 10))

sns.heatmap(heatmap_data, 
            annot=True, 
            fmt='.0f',
            cmap='RdYlGn_r',
            center=heatmap_data.values.mean(),
            cbar_kws={'label': 'Prix Médian (€)'},
            linewidths=0.5,
            ax=ax)

ax.set_title('Évolution des Prix Médians - Top 20 Départements', 
             fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('Année', fontsize=12)
ax.set_ylabel('Département', fontsize=12)

plt.tight_layout()
plt.show()

## 📊 8. Visualisations Interactives avec Plotly

In [ ]:
# Graphique interactif: Évolution temporelle
fig = px.line(
    agg_dvf[agg_dvf['Departement'].isin(top_20_depts.tolist()[:10])],
    x='year',
    y='prix_median',
    color='Departement',
    title='📈 Évolution des Prix Médians - Top 10 Départements (Interactif)',
    labels={'year': 'Année', 'prix_median': 'Prix Médian (€)'},
    markers=True,
    template='plotly_white'
)

fig.update_layout(
    hovermode='x unified',
    height=600,
    font=dict(size=12)
)

fig.show()

In [ ]:
# Graphique à barres animé
fig = px.bar(
    agg_dvf[agg_dvf['Departement'].isin(top_20_depts)],
    x='prix_median',
    y='Departement',
    animation_frame='year',
    orientation='h',
    color='prix_median',
    color_continuous_scale='Viridis',
    title='🏆 Classement des Départements par Prix Médian (Animation)',
    labels={'prix_median': 'Prix Médian (€)'},
    range_x=[0, agg_dvf['prix_median'].max() * 1.1],
    template='plotly_white'
)

fig.update_layout(
    height=700,
    yaxis={'categoryorder': 'total ascending'}
)

fig.show()

In [ ]:
# Scatter plot interactif animé
fig = px.scatter(
    agg_dvf,
    x='nb_transactions',
    y='prix_median',
    animation_frame='year',
    size='prix_median',
    color='Departement',
    hover_name='Departement',
    title='💰 Prix vs Volume de Transactions (Animation)',
    labels={
        'nb_transactions': 'Nombre de Transactions',
        'prix_median': 'Prix Médian (€)'
    },
    size_max=50,
    template='plotly_white'
)

fig.update_layout(height=600)
fig.show()

## 🔍 9. Analyse Statistique Approfondie

In [ ]:
# Matrice de corrélation
correlation_data = agg_dvf[['prix_median', 'prix_moyen', 'nb_transactions', 'prix_std']].corr()

fig, ax = plt.subplots(figsize=(10, 8))

sns.heatmap(correlation_data, 
            annot=True, 
            fmt='.3f',
            cmap='coolwarm',
            center=0,
            square=True,
            linewidths=1,
            cbar_kws={'label': 'Corrélation'},
            ax=ax)

ax.set_title('Matrice de Corrélation des Variables', 
             fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

print("\n📊 INTERPRÉTATION:")
print(f"Corrélation Prix Médian / Prix Moyen: {correlation_data.loc['prix_median', 'prix_moyen']:.3f}")
print(f"Corrélation Prix Médian / Transactions: {correlation_data.loc['prix_median', 'nb_transactions']:.3f}")

In [ ]:
# Volatilité des prix par département
volatilite = agg_dvf.groupby('Departement')['prix_median'].std().sort_values(ascending=False).head(15)

fig, ax = plt.subplots(figsize=(12, 6))

bars = ax.barh(range(len(volatilite)), volatilite.values, 
               color='coral', alpha=0.7, edgecolor='darkred')

ax.set_yticks(range(len(volatilite)))
ax.set_yticklabels(volatilite.index)
ax.set_xlabel('Écart-type du Prix Médian (€)', fontsize=12)
ax.set_title('Top 15 Départements - Volatilité des Prix (2020-2025)', 
             fontsize=14, fontweight='bold', pad=20)
ax.invert_yaxis()
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## 💾 10. Export des Données

In [ ]:
# Sauvegarde des données agrégées
agg_dvf.to_csv('donnees_aggregees_dvf.csv', index=False)
print("✅ Données agrégées exportées: donnees_aggregees_dvf.csv")

# Statistiques clés pour le dashboard
stats_cles = pd.DataFrame({
    'Métrique': [
        'Prix médian 2025',
        'Prix médian 2020',
        'Croissance totale (%)',
        'Croissance annuelle (%)',
        'Total transactions 2020-2025',
        'Nombre de départements'
    ],
    'Valeur': [
        f"{agg_dvf[agg_dvf['year']==2025]['prix_median'].mean():,.0f}€",
        f"{agg_dvf[agg_dvf['year']==2020]['prix_median'].mean():,.0f}€",
        f"{croissance_totale:.2f}%",
        f"{croissance_annuelle:.2f}%",
        f"{agg_dvf['nb_transactions'].sum():,}",
        f"{agg_dvf['Departement'].nunique()}"
    ]
})

stats_cles.to_csv('statistiques_cles.csv', index=False)
print("✅ Statistiques clés exportées: statistiques_cles.csv")

print("\n📊 STATISTIQUES CLÉS:")
print(stats_cles.to_string(index=False))

## 📝 11. Résumé et Conclusions

### 🎯 Principaux Constats

1. **Évolution des prix**: Le marché immobilier français a connu une croissance significative entre 2020 et 2025

2. **Disparités géographiques**: Paris et les départements de la petite couronne dominent largement le marché en termes de prix

3. **Volume de transactions**: Le volume varie selon les années, reflétant la dynamique du marché

4. **Volatilité**: Certains départements montrent une plus grande variabilité des prix que d'autres

### 💡 Prochaines Étapes

Pour enrichir cette analyse et créer votre dashboard, vous pouvez:

1. **Croiser avec d'autres données**:
   - Données démographiques (INSEE)
   - Infrastructures de transport (SNCF, Aéroports)
   - Établissements scolaires et services
   - Qualité de l'air et environnement

2. **Analyses complémentaires**:
   - Analyse par type de bien (maison, appartement)
   - Analyse par surface
   - Saisonnalité des transactions
   - Prédictions avec Machine Learning

3. **Dashboard interactif**:
   - Utiliser les templates Dash ou Streamlit fournis
   - Ajouter des filtres dynamiques
   - Créer des visualisations géographiques

---

**Bon courage pour la suite de votre projet ! 🚀**